In [1]:
# ---------------------------------------------
# PURPOSE: Real-time sign prediction using trained MLP model
# ---------------------------------------------

import cv2
import numpy as np
import os
from tensorflow.keras.models import load_model
import mediapipe as mp
from collections import deque

# Load MLP model
model = load_model("mlp_sign_model_finetuned.h5")
actions = ['hello', 'iloveyou', 'please', 'thankyou', 'ok', 'excuseme', 'sorry', 'welcome', 'stop', 'goodbye']
sequence_length = 30

# Load Mediapipe
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

# Extract Keypoints
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

# Real-time webcam
sequence = deque(maxlen=sequence_length)
cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = holistic.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION)
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

               # Show current model in use
        cv2.putText(image, 'Model: MLP', (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2, cv2.LINE_AA)

        keypoints = extract_keypoints(results)
        sequence.append(keypoints)

        if len(sequence) == sequence_length:
            input_data = np.array(sequence)
            input_data = (input_data - np.mean(input_data)) / np.std(input_data)
            input_data = input_data.reshape(1, sequence_length, -1)  # Match MLP input shape

            try:
                prediction = model.predict(input_data, verbose=0)[0]
                action = actions[np.argmax(prediction)]
                confidence = np.max(prediction)
                if confidence > 0.75:
                    cv2.putText(image, f'{action.upper()} ({confidence:.2f})', (10, 60),
                                cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 3, cv2.LINE_AA)
            except Exception as e:
                print("Prediction error:", e)


        cv2.imshow('MLP Sign Prediction', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

C:\Users\saite\AppData\Local\Temp\ipykernel_17892\2600450218.py:56: RuntimeWarning: invalid value encountered in divide
  input_data = (input_data - np.mean(input_data)) / np.std(input_data)
